In [1]:
import csv
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import time
from pprint import pprint
from datetime import datetime
import pytz
import shutil

In [2]:
version='23_24_gw3'
league_code = "551115"

In [3]:
start=time.perf_counter()

In [4]:
#Request league standings and return all league data 
def get_json_from_url(league_code):
    league_url="https://fantasy.premierleague.com/api/leagues-classic/"+league_code+"/standings/?page_new_entries=1&page_standings=1&phase=1"
    get=requests.get(league_url)
    league_data=json.loads(get.text)
    league_name = league_data['league'].get('name')
    print("Data fetched, league name: ", league_name)
    return league_data, league_name

#Use league data to return a dict of names & entry ids
def make_ids(league_data):
    member_ids={}
    for member in league_data['standings']['results']:
        name=member.get('player_name')
        mid=member.get('entry')
        member_ids[name]=mid
    return member_ids

#Use first value in member_ids to find the max gameweek that data is available for
def find_max_gw(xid): #xid=member entry id
    gw=1
    while gw < 100:
        try:
            get_mgw=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
            data_mgw=json.loads(get_mgw.text)
            list(data_mgw['entry_history'].keys())
            gw+=1
        except KeyError as err:
            print("Max gameweek data available for is: ",gw)
            return (gw)
            break

In [5]:
#(Doesn't work until GW1)
league_data, league_name = get_json_from_url(league_code)
member_ids=make_ids(league_data)
mgw=find_max_gw(list(member_ids.values())[0])

Data fetched, league name:  RIVERSIDE LEAGUE X 2023/24
Max gameweek data available for is:  4


In [6]:
#Get data1
def get_first_member_data(member_ids):
    xid=list(member_ids.values())[0]
    gw=1
    get_first=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
    data_first=json.loads(get_first.text)
    return data_first

#Member picks
def dataframe_from_ids(member_ids,cols):
    p=[]

     #member data

    for xid in member_ids.values():
        for gw in range(1,mgw):    
            get_gw=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
            data_gw=json.loads(get_gw.text)
            try:
                for x in range(0,15):
                    #picks
                    # print("Requesting...... ",xid,"GW "+str(gw),"Pick "+str(x+1))
                    row=data_gw["picks"][x]

                    name=list(member_ids.keys())[list(member_ids.values()).index(xid)]
                    selected_row = [name,xid,gw,x+1]

                    for i in cols:
                        selected_row.append(row.get(i))

                    p.append(selected_row)

                    print(xid,name,"GW "+str(gw),"Pick "+str(x+1))
            except KeyError as e:
                name = list(member_ids.keys())[list(member_ids.values()).index(xid)]
                print("GW Could not be found!",xid,name,"GW "+str(gw))
                print(e)
                continue
    print("Finished")
    pcols=['member','memberid','gw','pick']
    pcols.extend(cols)
    picksdf = pd.DataFrame(p, columns=pcols)
    return picksdf
data_for_columns=get_first_member_data(member_ids)
cols=list(data_for_columns["picks"][0].keys())
picksdf=dataframe_from_ids(member_ids,cols)

2654380 Joe Langwith GW 1 Pick 1
2654380 Joe Langwith GW 1 Pick 2
2654380 Joe Langwith GW 1 Pick 3
2654380 Joe Langwith GW 1 Pick 4
2654380 Joe Langwith GW 1 Pick 5
2654380 Joe Langwith GW 1 Pick 6
2654380 Joe Langwith GW 1 Pick 7
2654380 Joe Langwith GW 1 Pick 8
2654380 Joe Langwith GW 1 Pick 9
2654380 Joe Langwith GW 1 Pick 10
2654380 Joe Langwith GW 1 Pick 11
2654380 Joe Langwith GW 1 Pick 12
2654380 Joe Langwith GW 1 Pick 13
2654380 Joe Langwith GW 1 Pick 14
2654380 Joe Langwith GW 1 Pick 15
2654380 Joe Langwith GW 2 Pick 1
2654380 Joe Langwith GW 2 Pick 2
2654380 Joe Langwith GW 2 Pick 3
2654380 Joe Langwith GW 2 Pick 4
2654380 Joe Langwith GW 2 Pick 5
2654380 Joe Langwith GW 2 Pick 6
2654380 Joe Langwith GW 2 Pick 7
2654380 Joe Langwith GW 2 Pick 8
2654380 Joe Langwith GW 2 Pick 9
2654380 Joe Langwith GW 2 Pick 10
2654380 Joe Langwith GW 2 Pick 11
2654380 Joe Langwith GW 2 Pick 12
2654380 Joe Langwith GW 2 Pick 13
2654380 Joe Langwith GW 2 Pick 14
2654380 Joe Langwith GW 2 Pick 1

In [7]:
#Insert into PSQL
from sqlalchemy import create_engine, text
from datetime import datetime
import psycopg2 # python -> psql connection
import psycopg2.extras

#Import secret settings
import CJDH_local_settings
dbname = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['dbname']
user = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['user']
password = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['password']
host = "localhost"
port = 5432

#Create connection to psql via psycopg2 & sqlalchemy
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)
engine.autocommit = True  # read documentation understanding when to Use & NOT use # changes will be automatically committed to the database after each statement is executed.

In [8]:
picksdf.columns
picks = picksdf[['member', 'memberid', 'gw', 'pick', 'element', 'position', 'multiplier',
       'is_captain', 'is_vice_captain']]

In [9]:
#Create staging table for player data 
def create_staging_table_picks(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS picks CASCADE;
        CREATE UNLOGGED TABLE picks (
            member          TEXT,
            memberid        NUMERIC,
            gw              NUMERIC,
            pick            NUMERIC,
            element         NUMERIC,
            position        NUMERIC,
            multiplier      NUMERIC,
            is_captain      BOOLEAN,
            is_vice_captain BOOLEAN
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_picks(cursor)
picks.to_sql('picks', con=engine, if_exists='replace',index=False)

840

In [10]:
#Query gets only player details from the latest GW
query = """
        ALTER TABLE player_details
            RENAME COLUMN total_points TO total_points_gw;

        WITH playergw AS (
            SELECT *,
            ROW_NUMBER() OVER(
                    PARTITION BY
                        season,
                        code,
                        event,
                        fixture_id --Only one record per GW
                    ORDER BY 
                        now_datetime_utc DESC) as rn --Only select the latest date
        FROM playergw
        ),
        currentgw AS (
            SELECT MAX(current_gw) as max_gw
            FROM playergw
        )
        
        SELECT *
        FROM player_details
        LEFT JOIN playergw
            ON player_details.element = playergw.id
            AND player_details.round = playergw.current_gw
        CROSS JOIN currentgw
        WHERE player_details.round = currentgw.max_gw
            AND playergw.rn = 1
        ;
        """
with engine.connect() as conn:
    playerdf = pd.read_sql_query(query, conn)
print(playerdf.shape)

pd.pivot_table(playerdf, index=['id','web_name','team_name','round'],values='total_points_gw',aggfunc=np.sum).sort_values(by='total_points_gw', ascending=False)

(685, 133)


,,,,total_points_gw
id,web_name,team_name,round,
216,Sterling,Chelsea,3,19
36,Cash,Aston Villa,3,17
203,Gusto,Chelsea,3,14
599,Diaby,Aston Villa,3,12
519,Udogie,Spurs,3,12
...,...,...,...,...
686,Zych,Aston Villa,3,0
284,Rodák,Fulham,3,-1
313,Virgil,Liverpool,3,-2


In [11]:
max_gw = max(picksdf['gw'])
me = 'Christopher Harris'
myid = set(picksdf[picksdf['member']==me]['memberid'])
mypicksdf = picksdf[(picksdf['member']==me)&(picksdf['gw']==max_gw)]
mypicks = set(mypicksdf['element'])
mypicks

#league_standings
l = pd.DataFrame(league_data['standings']['results'])
mytotal = l[l['entry'].isin(myid)]['total'].values
#get ids for members above me or less than 100 points below me
ids_close_to_me = set(l[l['total']<=int(mytotal)+100]['entry'][~l['entry'].isin(myid)])

In [12]:
#Get ep_next * by ownership rate in mini-league
latestpicks = picksdf[(picksdf['gw']==max(picksdf['gw']))&(picksdf['memberid'].isin(ids_close_to_me))]
max_owns = len(set(latestpicks['memberid']))

ownership = pd.pivot_table(latestpicks, index=['element'],values='multiplier',aggfunc=np.sum).reset_index()
ownership['own_rate'] = ownership['multiplier']/max_owns

ownership.sort_values(by='own_rate', ascending=False)

playerdf = pd.merge(playerdf, ownership, how='outer',on='element')
filtered = playerdf['element'].isin(mypicks)
gainpicks = playerdf[filtered].merge(mypicksdf,how='left',on='element',suffixes=('','_myteam'))

gainpicks['expected_gain'] = gainpicks['ep_next'].astype(float)*(gainpicks['multiplier_myteam']-gainpicks['own_rate'])
gainpicks[['web_name','element','element_type','multiplier','multiplier_myteam','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False)

,web_name,element,element_type,multiplier,multiplier_myteam,own_rate,ep_next,expected_gain
9,Foden,353,3,3.0,1,0.166667,6.3,5.250000
3,Watkins,60,4,3.0,1,0.166667,4.5,3.750000
8,T.Silva,217,2,1.0,1,0.055556,3.0,2.833333
7,Chilwell,195,2,8.0,1,0.444444,4.7,2.611111
14,Onana,597,1,7.0,1,0.388889,3.7,2.261111
11,Rashford,396,3,8.0,1,0.444444,4.0,2.222222
0,Martinelli,12,3,6.0,1,0.333333,3.3,2.200000
10,Haaland,355,4,31.0,2,1.722222,7.3,2.027778
1,Saka,19,3,13.0,1,0.722222,7.0,1.944444
4,Estupiñan,131,2,13.0,1,0.722222,5.8,1.611111


In [13]:
print("Expected Total Gain on Division Average")
gainpicks['expected_gain'].sum()

Expected Total Gain on Division Average


27.5

In [14]:
#players not in my picks
losspicks = playerdf[~filtered].copy().dropna(subset='multiplier')
losspicks['expected_loss'] = losspicks['ep_next'].astype(float)*(0-losspicks['own_rate'])
print("Expected Total Loss on Division Average: ")
print(losspicks['expected_loss'].sum())
losspicks[['web_name','element','multiplier','own_rate','ep_next','expected_loss']].sort_values(by='expected_loss',ascending=True).head()

Expected Total Loss on Division Average: 
-25.65


,web_name,element,multiplier,own_rate,ep_next,expected_loss
107,Mbeumo,108,5.0,0.277778,8.3,-2.305556
307,Salah,308,6.0,0.333333,5.5,-1.833333
372,B.Fernandes,373,5.0,0.277778,5.3,-1.472222
19,Saliba,20,5.0,0.277778,4.7,-1.305556
293,Diogo J.,294,3.0,0.166667,6.2,-1.033333


In [15]:
print("Net Gain")
print(round(gainpicks['expected_gain'].sum() + losspicks['expected_loss'].sum(),2))

Net Gain
1.85


In [16]:
teamgains = gainpicks[['web_name','status','element','element_type','now_cost','multiplier','multiplier_myteam','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False)
teamgains['expected_loss_on_transfer'] = teamgains['ep_next'].astype(float)*(teamgains['own_rate'])

allgains = playerdf.copy()
allgains['expected_gain'] = allgains['ep_next'].astype(float)*(1-allgains['own_rate'])

def top_suggested_transfer(row, df):
    #get element, element_type & gain
    myteam_type = row['element_type']
    # myteam_gain = row['expected_gain']
    myteam_loss = row['expected_loss_on_transfer']
    myteam_cost = row['now_cost']
    myteam_element = row['element']
    #get playerdf element, element_type & gain
    filtered = (df['element_type']==myteam_type) & (df['expected_gain']>myteam_loss) & (df['now_cost']<=myteam_cost) & (df['element']!=myteam_element)
    best_gain = df[filtered].sort_values(by='expected_gain', ascending=False).head(1)
    best_gain['own_rate'] = round(best_gain['own_rate'],3)
    best_gain['expected_gain'] = round(best_gain['expected_gain'],2)
    best_gain['diff'] = round(df['expected_gain']-row['expected_loss_on_transfer'],2)
    return best_gain[['web_name','status','own_rate','ep_next','expected_gain','diff']].values[0]

teamgains['affordable_suggested_transfer'] = teamgains.apply(lambda x: top_suggested_transfer(x, allgains),axis=1)
# Extracting the second element from the arrays in the "affordable_suggested_transfer" column
teamgains['suggested_transfer_gain'] = teamgains['affordable_suggested_transfer'].apply(lambda x: x[-1])

# Sorting the DataFrame based on the second element in the arrays
teamgains = teamgains.sort_values(by='suggested_transfer_gain', ascending=False)
teamgains.head(20)

,web_name,status,element,element_type,now_cost,multiplier,multiplier_myteam,own_rate,ep_next,expected_gain,expected_loss_on_transfer,affordable_suggested_transfer,suggested_transfer_gain
9,Foden,a,353,3,76,3.0,1,0.166667,6.3,5.250000,1.050000,"[Rodrigo, a, 0.056, 9.7, 9.16, 8.11]",8.11
0,Martinelli,a,12,3,80,6.0,1,0.333333,3.3,2.200000,1.100000,"[Rodrigo, a, 0.056, 9.7, 9.16, 8.06]",8.06
11,Rashford,a,396,3,90,8.0,1,0.444444,4.0,2.222222,1.777778,"[Rodrigo, a, 0.056, 9.7, 9.16, 7.38]",7.38
2,Turner,a,28,1,40,1.0,0,0.055556,2.3,-0.127778,0.127778,"[Areola, a, 0.0, 6.5, 6.5, 6.37]",6.37
8,T.Silva,a,217,2,50,1.0,1,0.055556,3.0,2.833333,0.166667,"[Cash, a, 0.056, 6.8, 6.42, 6.26]",6.26
3,Watkins,a,60,4,80,3.0,1,0.166667,4.5,3.750000,0.750000,"[Wissa, a, 0.056, 7.3, 6.89, 6.14]",6.14
14,Onana,a,597,1,50,7.0,1,0.388889,3.7,2.261111,1.438889,"[Areola, a, 0.0, 6.5, 6.5, 5.06]",5.06
5,Mitoma,a,143,3,66,15.0,1,0.833333,5.5,0.916667,4.583333,"[Rodrigo, a, 0.056, 9.7, 9.16, 4.58]",4.58
7,Chilwell,a,195,2,58,8.0,1,0.444444,4.7,2.611111,2.088889,"[Cash, a, 0.056, 6.8, 6.42, 4.33]",4.33
1,Saka,a,19,3,86,13.0,1,0.722222,7.0,1.944444,5.055556,"[Rodrigo, a, 0.056, 9.7, 9.16, 4.11]",4.11


In [17]:
print("Sorted by Expected Gain")
allgains[['web_name','status','element','element_type','now_cost','ep_next','own_rate','expected_gain']].sort_values(by='expected_gain',ascending=False).dropna().head(20)

Sorted by Expected Gain


,web_name,status,element,element_type,now_cost,ep_next,own_rate,expected_gain
364,Rodrigo,a,365,3,56,9.7,0.055556,9.161111
139,March,a,140,3,66,8.2,0.055556,7.744444
662,Ward-Prowse,a,664,3,60,8.0,0.055556,7.555556
503,Maddison,a,504,3,76,7.8,0.111111,6.933333
118,Wissa,a,119,4,62,7.3,0.055556,6.894444
523,Areola,a,524,1,40,6.5,0.000000,6.500000
598,Diaby,a,599,3,66,6.8,0.055556,6.422222
35,Cash,a,36,2,45,6.8,0.055556,6.422222
107,Mbeumo,a,108,3,67,8.3,0.277778,5.994444
522,Antonio,a,523,4,60,6.2,0.055556,5.855556


In [18]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#17/08/23: Finished in 0.43 minute(s)

Finished in 0.67 minute(s)
